In [2]:
import requests, time
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup
import re
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from tqdm import tqdm
import folium
import math

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
service = Service('data/chromedriver_win32/chromedriver.exe')
driver = webdriver.Chrome(service=service)

In [4]:
base_url= 'https://kin.naver.com/search/list.naver?query=%EC%96%B4%EB%B2%84%EC%9D%B4%EB%82%A0%20%EC%84%A0%EB%AC%BC'
# url = f'{base_url} + {quote("어버이날 선물")}'
url = f'{base_url}'
driver.get(url)
soup = BeautifulSoup(driver.page_source,'html.parser')
time.sleep(2)

In [5]:
lis = soup.select('ul.basic1 > li')
len(lis)

10

In [6]:
li = lis[0]
sub_url = li.select_one('a._searchListTitleAnchor')['href']
sub_url

'https://kin.naver.com/qna/detail.naver?d1id=8&dirId=81105&docId=443561769&qb=7Ja067KE7J2064KgIOyEoOusvA==&enc=utf8&section=kin&rank=1&search_sort=0&spq=0'

In [7]:
driver.get(sub_url)
time.sleep(2)

In [8]:
count = int(driver.find_element(By.XPATH,'//*[@id="answerArea"]/div/div[1]/div[1]/h3/em').text)
page_count = (count - 1) // 5          # 5까지 첫번째 페이지, 첫페이지까진 next page 안누름
print(count, page_count)

2 0


In [9]:
for i in range(page_count):
    driver.find_element(By.ID,'nextPageButton').click()
    time.sleep(2)

In [10]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
divs = soup.select('div.answer-content__item._contentWrap._answer')
len(divs)

2

In [11]:
div = divs[0]
ps = div.select('p.se-text-paragraph')
for p in ps:
    print(p.get_text())

안녕하세요.
부모님께서 어깨랑 무릎이 아프시다고 하셨는데, 저희 부모님께서 사용하시면서
매번 칭찬하시는 제품이 생각나서 답글 남겨요~.
​
원적외선 온열치료기인데, 일반적인 적외선 조사기 처럼 생겼는데 신기하게 빨간 빛이 안나는 제품이에요. 
램프가 세라믹으로 만들어 진 것이라 눈에 보이는 빨간 빛이 없이, 열감만 나는 제품인데요, 
이게 파장이 일반 적외선 조사기랑 달라요. 파장이 몸속 깊숙이 침투되는 생육광선 파장을 낸다고 해요. 
그래서 빨간 빛이 없고, 사용했을때 확실히 열감이 다르다고 하세요. 
저희 부모님도 이거 매일 사용하시고 어깨 허리 무릎 많이 좋아지셨어요.
​
조금 피로하신 날에도 어깨랑 등이랑 자주 사용하시는데 그러면 사우나 한 것 처럼 개운하다고 하시더라구요~.
​
두 분이 서로 사용하시겠다고 하셔서;;; 하나 더 사드려야 하나 하던 참이었어요;;
​
여튼, 조작법도 쉽고, 사실 여기저기 사용 부위 제한도 없고, 또 제품도 엄청 튼튼해보여요.
후기 보니까 부모님 선물로 많이 드리고, 만족도도 높더라구요!
​
가격도 30만원 조금 안되는 가격이라 원하시는 가격대에 딱 맞네요 ^^
좋은 선물 하시기 바래요~!!
​
도움이 되셨다면 꼭 채택 부탁드립니다! ^^
​


In [12]:
driver.back()
time.sleep(2)

In [13]:
url

'https://kin.naver.com/search/list.naver?query=%EC%96%B4%EB%B2%84%EC%9D%B4%EB%82%A0%20%EC%84%A0%EB%AC%BC'

In [14]:
lines = []
for page in tqdm(range(1,11)):
    url = f"{base_url}&page={page}"
    driver.get(url)
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select('ul.basic1 > li')
    for li in lis:
        sub_url = li.select_one('a._searchListTitleAnchor')['href']
        driver.get(sub_url)
        time.sleep(2)

        count = int(driver.find_element(By.XPATH,'//*[@id="answerArea"]/div/div[1]/div[1]/h3/em').text)
        page_count = (count - 1) // 5
        # print(count, end=', ')
        for i in range(page_count):
            driver.find_element(By.ID,'nextPageButton').click()
            time.sleep(2)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        divs = soup.select('div.answer-content__item._contentWrap._answer')
        print(len(divs))
        for div in divs:
            ps = div.select('p.se-text-paragraph')
            ans_text_str = ''
            for p in ps:
                ans_text = p.get_text()
                if ans_text == ' ' or ans_text == '\u200b':         # zero width space
                    continue
                ans_text_str += ans_text + '\n'
            lines.append(ans_text_str)

    driver.back()
    time.sleep(2)

  0%|          | 0/10 [00:00<?, ?it/s]

2
2
7
7
8
21
7
5
3
10


 10%|█         | 1/10 [00:53<08:05, 53.99s/it]

6
16
12
12
2
6
10
7
1
10


 20%|██        | 2/10 [01:56<07:54, 59.27s/it]

...

 80%|████████  | 8/10 [08:14<02:02, 61.24s/it]

12
10
6
11
12
6
10
6
10
9


 90%|█████████ | 9/10 [09:28<01:05, 65.11s/it]

12
13
10
4
9
9
2
2
5
14


100%|██████████| 10/10 [10:34<00:00, 63.47s/it]


In [15]:
driver.close()

In [19]:
len(lines)

801

In [17]:
str_line = ''.join([str(line) + '\n' for line in lines])
str_line

'안녕하세요.\n부모님께서 어깨랑 무릎이 아프시다고 하셨는데, 저희 부모님께서 사용하시면서\n매번 칭찬하시는 제품이 생각나서 답글 남겨요~.\n원적외선 온열치료기인데, 일반적인 적외선 조사기 처럼 생겼는데 신기하게 빨간 빛이 안나는 제품이에요. \n램프가 세라믹으로 만들어 진 것이라 눈에 보이는 빨간 빛이 없이, 열감만 나는 제품인데요, \n이게 파장이 일반 적외선 조사기랑 달라요. 파장이 몸속 깊숙이 침투되는 생육광선 파장을 낸다고 해요. \n그래서 빨간 빛이 없고, 사용했을때 확실히 열감이 다르다고 하세요. \n저희 부모님도 이거 매일 사용하시고 어깨 허리 무릎 많이 좋아지셨어요.\n조금 피로하신 날에도 어깨랑 등이랑 자주 사용하시는데 그러면 사우나 한 것 처럼 개운하다고 하시더라구요~.\n두 분이 서로 사용하시겠다고 하셔서;;; 하나 더 사드려야 하나 하던 참이었어요;;\n여튼, 조작법도 쉽고, 사실 여기저기 사용 부위 제한도 없고, 또 제품도 엄청 튼튼해보여요.\n후기 보니까 부모님 선물로 많이 드리고, 만족도도 높더라구요!\n가격도 30만원 조금 안되는 가격이라 원하시는 가격대에 딱 맞네요 ^^\n좋은 선물 하시기 바래요~!!\n도움이 되셨다면 꼭 채택 부탁드립니다! ^^\n\n어버이날 선물 드리려고 하시는 군요 ㅎㅎ\n확실히 부모님들 나이가 들면서\n점점 아파하고 뻐근해 하시는게 눈에 보이죠..\n확실히 현금으로만 드리는거는 좀 감동이 적어서\n어버이날 선물 잘 주시는게 좋아요\n어버이날 선물로 편지와 현금과 함께\n건강보조식품이나 보조기구를 주시면 좋을것 같은데\n아무래도 예산이 적다보니 꼭 후기를 많이 보고\n잘 맞는지 고민을 해보시고 A/S는 되는지\n꾸준히 쓸만한 제품인지, 너무 뻔하지 않은지 등\n고려해서 어버이날 선물 주시면 좋겠네요!\n\n안녕하세요!!\n저도 작년에 같은 고민하다가 선물로 드린\n휴그랩 손마사지기 생각나서 답변 남겨요 ㅎㅎ\n작년에 받으시고 올해까지 계속 칭찬하셔서 이번엔\n뭘 해드려야하나 전 그게 고민이네요..ㅠㅜ\n

In [21]:
with open('data/네이버지식인.txt','w',encoding='utf-8') as f:
    f.write(str_line)